In [1]:
# this should look familiar from the previous labs. 
# We could choose any of the pre-built nets here instead.
import os
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Nadam
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage

# This is a new built in dataset that we haven't seen before.
# It is 60,000 (50k training, 10k test) small (32x32) RGB images 
# classified into 100 classes:
from tensorflow.keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

# Constant number of labels, square image shape
NUM_CLASSES = 100
IMAGE_SIZE = 96

# FOR DISPLAY PURPOSES
unprocessed_training_images = x_train
unprocessed_training_labels = y_train

# Because the network we're fine-tuning (MobileNetV2) has several pooling layers
# the smallest image it can process is 96x96, these images are 32x32. To fix this
# we are manually rescaling all the images using scipy. We are also applying the 
# MobileNetV2 preprocess_input function here. 
def adjust_input_image(rgb_data):
    adjusted = preprocess_input(rgb_data)
    
    # Scales width and height by 3, leaves color channels at original scale
    adjusted = ndimage.zoom(adjusted, (3, 3, 1), order=0)

    return adjusted

x_train = np.array([adjust_input_image(x) for x in x_train])
x_test = np.array([adjust_input_image(x) for x in x_test])

# And we still need to one-hot encode the labels as usual
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [2]:
# This should look familiar from previous labs.
def plot_training_history(history, model):
    figure = plt.figure()

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.tight_layout()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.tight_layout()

    figure.tight_layout()
    plt.show()

In [3]:
# The first way we'll do transfer learning REALLY saves time, but
# only works if you don't want to fine tune any of the layers 
# from the original network. What we can do instead is memorize
# the output of the base network, since it won't be changing,
# and then repeatedly train on those outputs and the training labels

# Like before, we grab a pretrained model with include_top=False
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# Unlike before, we're going to just run the images through this base layer once
# This takes awhile, we're essentially doing a round of evaluation on both datasets.
# And we'll save them incase we want to experiment with different models to transfer
# on top of these.
training_features = base_model.predict(x_train)
np.savez('MobileNetV2_features_train', features=training_features)

test_features = base_model.predict(x_test)
np.savez('MobileNetV2_features_test', features=test_features)

In [4]:
def discriminative_fine_tuning(model, optimizer, layer_unfreeze_points, epochs_per_freeze_point, batch_size):
    # All histories returned for more holistic visualiation after the fact.
    all_histories = []
    
    # Intially freeze everything, we'll unfreeze layers iteratively as we train
    for layer in model.layers:
        layer.trainable = False
    
    # Caller specifies blocks of layers to unfreeze at once
    for current_unfreeze_point in layer_unfreeze_points:
        
        # Unfreeze everything after the current freeze point
        print("Unfreezing layers after: ", current_unfreeze_point)
        for layer in model.layers[current_unfreeze_point:]:
            layer.trainable = True
        
        # Must compile after freezing/unfreezing or the changes won't be applied
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Train at each unfreeze point
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs_per_freeze_point, validation_split=0.2, verbose=True)
        all_histories.append(history)
    
    return all_histories, model


def plot_combined_histories(all_histories):
    # Some code to plot all the histories at once...
    acc = []
    val_acc = []
    loss = []
    val_loss = []

    # Simply cobble together the individual histories
    for history in all_histories:
        acc.extend(history.history['accuracy'])
        val_acc.extend(history.history['val_accuracy'])
        loss.extend(history.history['loss'])
        val_loss.extend(history.history['val_loss'])

    # And plot them the same way as the function at the top of this notebook.
    figure = plt.figure()

    plt.subplot(1, 2, 1)
    plt.plot(acc)
    plt.plot(val_acc)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.tight_layout()

    plt.subplot(1, 2, 2)
    plt.plot(loss)
    plt.plot(val_loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.tight_layout()

    figure.tight_layout()
    plt.show()

In [5]:
def discriminative_fine_tuning_decay(model, layer_unfreeze_points, epochs_per_freeze_point, batch_size, lr=.001):
    # All histories returned for more holistic visualiation after the fact.
    all_histories = []
    
    # Intially freeze everything, we'll unfreeze layers iteratively as we train
    for layer in model.layers:
        layer.trainable = False
    
    # Caller specifies blocks of layers to unfreeze at once
    for current_unfreeze_point in layer_unfreeze_points:
        
        # Unfreeze everything after the current freeze point
        print("Unfreezing layers after: ", current_unfreeze_point)
        for layer in model.layers[current_unfreeze_point:]:
            layer.trainable = True
        
        # Must compile after freezing/unfreezing or the changes won't be applied
        print("Learning Rate: ", lr)
        optimizer = Nadam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        lr *= 0.80
        
        # Train at each unfreeze point
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs_per_freeze_point, validation_split=0.2, verbose=True)
        all_histories.append(history)
    
    return all_histories, model

In [6]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

old_top = base_model.output
old_top = GlobalAveragePooling2D()(old_top)
old_top = Dense(units=640, activation='relu')(old_top)
old_top = Dropout(rate=0.4)(old_top)
old_top = Dense(units=320, activation='relu')(old_top)
old_top = Dropout(rate=0.4)(old_top)
new_top = Dense(NUM_CLASSES, activation='softmax')(old_top)

model = Model(inputs=base_model.input, outputs=new_top)

# These points were carefully chosen based on the model.
# Specifially, each convolutional bottleneck block is unfrozen as a whole
# Note: Descending order is required for the above funtion to work as expected
layer_unfreeze_points = [
 #   154, 
 #   144, 
 #   126, 
 #   108,
 #   99,
 #   91,
 #   81,
    73
]

# Some changes here...
all_histories, model = discriminative_fine_tuning_decay(model, layer_unfreeze_points, 20, 128)
model.save(os.path.join(save_directory, "discriminative_half_decay_ten_epoch.h5"))
plot_combined_histories(all_histories)

Unfreezing layers after:  73
Learning Rate:  0.001
Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 64s 2ms/sample - loss: 2.4602 - accuracy: 0.3965 - val_loss: 4.2137 - val_accuracy: 0.1297
Epoch 2/20
40000/40000 [==============================] - 48s 1ms/sample - loss: 1.5626 - accuracy: 0.5927 - val_loss: 3.7833 - val_accuracy: 0.1599
Epoch 3/20
40000/40000 [==============================] - 43s 1ms/sample - loss: 1.2641 - accuracy: 0.6606 - val_loss: 3.9745 - val_accuracy: 0.1483
Epoch 4/20
40000/40000 [==============================] - 43s 1ms/sample - loss: 1.0797 - accuracy: 0.7044 - val_loss: 4.1716 - val_accuracy: 0.1066
Epoch 5/20
40000/40000 [==============================] - 45s 1ms/sample - loss: 0.9438 - accuracy: 0.7423 - val_loss: 3.9571 - val_accuracy: 0.1663
Epoch 6/20
40000/40000 [==============================] - 45s 1ms/sample - loss: 0.8342 - accuracy: 0.7716 - val_loss: 4.0626 - val_accuracy: 0.1593
Epoch

NameError: name 'save_directory' is not defined